# [Internal Only] CSE6242 Team 11 Colab
## This colab is used to join/clean/analysis our data



In [2]:
%rm -rf PawsHelper

In [3]:
!git clone https://github.com/ChujieChen/PawsHelper.git

Cloning into 'PawsHelper'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 67 (delta 26), reused 41 (delta 11), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [4]:
%cd PawsHelper/src

/content/PawsHelper/src


In [5]:
import pandas as pd
import seaborn as sns
import numpy as np

In [6]:
df_intakes = pd.read_csv("../data/Austin_Animal_Center_Intakes.csv")
df_outcomes = pd.read_csv("../data/Austin_Animal_Center_Outcomes.csv")

In [7]:
df_intakes.head(3)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White


In [8]:
df_outcomes.head(3)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,08/16/2020 11:38:00 AM,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray


In [9]:
df_intakes['Intake Type'].unique()

array(['Stray', 'Owner Surrender', 'Public Assist', 'Wildlife',
       'Euthanasia Request', 'Abandoned'], dtype=object)

In [10]:
df_outcomes['Outcome Type'].unique()

array(['Rto-Adopt', 'Adoption', 'Euthanasia', 'Transfer',
       'Return to Owner', 'Died', 'Disposal', 'Missing', 'Relocate', nan],
      dtype=object)

# Some repetitive `Animal ID`, what do we do?
Step1: 
Vertically stack intakes and outcomes (130k rows + 130k rows = 265k rows)
Action_type = intake_type U outcome_type (9 types)
其他对于intakes里有而outcomes里没有的column，outcome那列那行为空；反之亦然
e.g.
Animal ID，datetime, action_type, XXX, XXX, XXX\
A12345, 08/09/2019, intake,           aa,  aa, aa\
A12345, 09/09/2019, outcome,           aa,  aa, aa\
A12345, 10/11/2019, outcome,           aa,  aa, aa\
A12345, 10/12/2019, intake,           aa,  aa, aa\
A12345, 10/15/2019, outcome,           aa,  aa, aa\
A12345, 05/04/2020, intake ,          aa, aa, aa\

上述例子，A12345被两次intake，两次outcome；A23456，只有intake，尚无outcome



In [11]:
df_join = df_intakes.append(df_outcomes, sort=False)

In [12]:
mask = df_join['Intake Type'].isna()
df_join['action_type'] = "nan"
df_join['action_type'][mask] = "outcome"
df_join['action_type'][~mask] = "intake"


In [13]:
df_join = df_join.reset_index(drop=True)

In [14]:
df_join

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome,action_type
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,NaN,NaN,NaN,NaN,NaN,intake
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,NaN,NaN,NaN,NaN,NaN,intake
2,A724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,NaN,NaN,NaN,NaN,NaN,intake
3,A665644,NaN,10/21/2013 07:59:00 AM,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,NaN,NaN,NaN,NaN,NaN,intake
4,A682524,Rio,06/29/2014 10:38:00 AM,06/29/2014 10:38:00 AM,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,NaN,NaN,NaN,NaN,NaN,intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265405,A843114,A843114,10/20/2021 06:02:00 PM,10/20/2021 06:02:00 PM,NaN,NaN,NaN,Cat,NaN,NaN,Domestic Shorthair,Torbie,09/27/2019,Transfer,Partner,Spayed Female,2 years,outcome
265406,A844350,*Ella,10/20/2021 06:51:00 PM,10/20/2021 06:51:00 PM,NaN,NaN,NaN,Cat,NaN,NaN,Domestic Shorthair,Brown Tabby,04/15/2021,Adoption,NaN,Spayed Female,6 months,outcome
265407,A844415,A844415,10/20/2021 06:01:00 PM,10/20/2021 06:01:00 PM,NaN,NaN,NaN,Cat,NaN,NaN,Domestic Shorthair Mix,Silver Tabby,07/31/2021,Adoption,NaN,Spayed Female,2 months,outcome
265408,A844004,*Lestat,10/20/2021 06:39:00 PM,10/20/2021 06:39:00 PM,NaN,NaN,NaN,Dog,NaN,NaN,Siberian Husky,Black/White,05/10/2020,Adoption,NaN,Neutered Male,1 year,outcome


In [15]:
df_join.columns

Index(['Animal ID', 'Name', 'DateTime', 'MonthYear', 'Found Location',
       'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake',
       'Age upon Intake', 'Breed', 'Color', 'Date of Birth', 'Outcome Type',
       'Outcome Subtype', 'Sex upon Outcome', 'Age upon Outcome',
       'action_type'],
      dtype='object')

In [16]:
df_join['DateTime'] = pd.to_datetime(df_join['DateTime']) # make sure DateTime is datetime

In [17]:
df_join_sorted = df_join.sort_values(by = ['Animal ID', 'DateTime'])

# Step2:
Window function：对同一只动物（Animal ID），内部sort Datatime
有进有出的动物，则两相邻行的Datetime合为一行，其他列，用有数据的那行的数据
最后行数应为13万到26万之间的一个行数，同一 e.g
Animal ID，datetime_intake, datetime_outcome, xxx, xxx, xxx
A12345,      08/09/2019      ,    09/09/2019         , aaa ,aa, aa
A12345,      10/12/2019      ,    10/15/2019         , aaa ,aa, aa
A23456,        05/04/2019,                                     , aaa, aa, aa




In [18]:
df_join_sorted = df_join_sorted.reset_index(drop=True)

In [19]:
prev = ""
prevId = ""
data_in_out = []
data_row = []
incolumns = ['Animal ID', 'Name', 'DateTime', 'Found Location',
        'Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake',
        'Age upon Intake', 'Breed', 'Color']

outcolumns = ['DateTime', 'Date of Birth', 'Outcome Type',
       'Outcome Subtype', 'Sex upon Outcome', 'Age upon Outcome']
for idx, row in df_join_sorted.iterrows():
  t, tid = row['action_type'], row['Animal ID']
  if tid == prevId and t == "outcome" and prev == "intake":
      # combine and append
      data_row.extend(row[outcolumns].values.tolist())
      data_in_out.append(data_row)
      data_row = []
  else:
      if len(data_row) != 0:
        if prev == "intake":
          tmp = df_join_sorted.iloc[idx-1][outcolumns].values.tolist()
          tmp[0] = np.nan
          data_row.extend(tmp)
        data_in_out.append(data_row)
        data_row = []
      
      if t == "intake":
        data_row = row[incolumns].values.tolist()
      elif t == "outcome":
        tmp = row[incolumns].values.tolist()
        tmp[2] = np.nan
        data_row = tmp
        data_row.extend(row[outcolumns].values.tolist())
        data_in_out.append(data_row)
        data_row = []
      # anyways, we need incolumns
  prev, prevId = t, tid
if len(data_row) != 0:
  data_row.extend(df_join_sorted.iloc[-1][outcolumns].values.tolist())
  data_in_out.append(data_row)

df_in_out = pd.DataFrame(data_in_out, columns=['animal_id', 'name', 'intake_datetime', 'found_location',
        'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake',
        'age_upon_intake', 'breed', 'color','outcome_datetime', 'date_of_birth', 'outcome_type',
       'outcome_subtype', 'sex_upon_outcome', 'age_upon_outcome'])

In [20]:
# df_in_out.to_csv("data/Austin_Animal_Center_Joined.csv", index=False)

# Data Clean

In [21]:
df_join = pd.read_csv("../data/Austin_Animal_Center_Joined.csv")

In [22]:
df_join.describe()

,animal_id,name,intake_datetime,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,outcome_datetime,date_of_birth,outcome_type,outcome_subtype,sex_upon_outcome,age_upon_outcome
count,134227,93225,132678,132678,132678,132678,134227,132677,132678,134227,134227,132733,132732,132707,60955,132731,132717
unique,119460,22148,93276,56368,6,15,5,5,52,2698,609,110115,7397,9,24,5,52
top,A721033,Max,2016-09-23 12:00:00,Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Domestic Shorthair Mix,Black/White,2016-04-18 00:00:00,04/21/2014,Adoption,Partner,Neutered Male,1 year
freq,33,604,64,24823,91756,114514,75255,43460,22805,32362,14027,39,117,60110,32595,46676,23162


In [23]:
df_join.apply(lambda x: sum(x.isnull()/len(df_join)))

animal_id           0.000000
name                0.305468
intake_datetime     0.011540
found_location      0.011540
intake_type         0.011540
intake_condition    0.011540
animal_type         0.000000
sex_upon_intake     0.011548
age_upon_intake     0.011540
breed               0.000000
color               0.000000
outcome_datetime    0.011130
date_of_birth       0.011138
outcome_type        0.011324
outcome_subtype     0.545881
sex_upon_outcome    0.011145
age_upon_outcome    0.011250
dtype: float64

In [24]:
df_clean = df_join[(~df_join.sex_upon_intake.isnull()) | (~df_join.intake_datetime.isnull())]

In [25]:
df_clean.apply(lambda x: sum(x.isnull()/len(df_clean)))

animal_id           0.000000
name                0.305145
intake_datetime     0.000000
found_location      0.000000
intake_type         0.000000
intake_condition    0.000000
animal_type         0.000000
sex_upon_intake     0.000008
age_upon_intake     0.000000
breed               0.000000
color               0.000000
outcome_datetime    0.011260
date_of_birth       0.011268
outcome_type        0.011441
outcome_subtype     0.548154
sex_upon_outcome    0.011275
age_upon_outcome    0.011381
dtype: float64

In [26]:
df_clean['intake_datetime'] = pd.to_datetime(df_clean['intake_datetime'])
df_clean['outcome_datetime'] = pd.to_datetime(df_clean['outcome_datetime'])
df_clean['date_of_birth'] = pd.to_datetime(df_clean['date_of_birth'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [32]:
df_clean.loc[df_clean['name'].isnull(), "has_name"] = 0
df_clean.loc[df_clean['name'] == df_clean['animal_id'], "has_name"] = 0
df_clean.loc[~df_clean['name'].isnull(), "has_name"] = 1

In [33]:
def getBirthSex(sex_upon_intake):
  sex_upon_intake = str(sex_upon_intake)
  if "Male" in sex_upon_intake:
    return "Male"
  elif "Female" in sex_upon_intake:
    return "Female"
  return np.nan

def isPureColor(color):
  color = str(color)
  if color != "nan":
    color_list = color.split("/")
    if "Tricolor" in color_list:
      return 0
    if len(color_list) > 1:
      return 0
    else:
      return 1
  else:
    return np.nan


def sex_changed(sex_upon_outcome):
  sex = str(sex_upon_outcome).lower()
  if ("splayed" in sex) or ("neutered" in sex):
    return 1
  elif ("unknown" in sex) or ("nan" in sex):
    return np.nan
  return 0


df_clean['age_upon_intake_day'] = (df_clean['intake_datetime'] - df_clean['date_of_birth']).dt.days
df_clean['age_upon_outcome_day'] = (df_clean['outcome_datetime'] - df_clean['date_of_birth']).dt.days
# 168 rows having negatives
df_clean = df_clean[(df_clean.age_upon_intake_day >=0) | (df_clean.age_upon_intake_day.isnull()) ]
df_clean = df_clean[(df_clean.age_upon_outcome_day >=0) | (df_clean.age_upon_outcome_day.isnull()) ]
df_clean['birth_sex'] = df_clean.sex_upon_intake.apply(getBirthSex)
df_clean['pure_color'] = df_clean.color.apply(isPureColor)
df_clean['sex_changed'] = df_clean.sex_upon_outcome.apply(sex_changed)

In [29]:
df_clean.describe()

,has_name,age_upon_intake_day,age_upon_outcome_day,pure_color,sex_changed
count,132508.000000,131013.000000,131013.000000,132508.000000,120537.000000
mean,0.695113,780.936220,799.744766,0.445317,0.382920
std,0.460361,1069.276596,1072.163480,0.497003,0.486101
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,74.000000,94.000000,0.000000,0.000000
50%,1.000000,365.000000,370.000000,0.000000,0.000000
75%,1.000000,904.000000,923.000000,1.000000,1.000000
max,1.000000,9132.000000,9137.000000,1.000000,1.000000


In [30]:
df_clean.apply(lambda x: sum(x.isnull()/len(df_clean)))

animal_id               0.000000
name                    0.304887
intake_datetime         0.000000
found_location          0.000000
intake_type             0.000000
intake_condition        0.000000
animal_type             0.000000
sex_upon_intake         0.000008
age_upon_intake         0.000000
breed                   0.000000
color                   0.000000
outcome_datetime        0.011275
date_of_birth           0.011282
outcome_type            0.011456
outcome_subtype         0.548390
sex_upon_outcome        0.011290
age_upon_outcome        0.011396
has_name                0.000000
age_upon_intake_day     0.011282
age_upon_outcome_day    0.011282
birth_sex               0.080833
pure_color              0.000000
sex_changed             0.090342
dtype: float64

In [34]:
df_clean['intake_times'] = 0
df_clean['intake_times'] = df_clean.sort_values(['animal_id','intake_datetime'], ascending=[True,True]) \
             .groupby(['animal_id']) \
             .cumcount() + 1

In [35]:
df_clean.head()

,animal_id,name,intake_datetime,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,outcome_datetime,date_of_birth,outcome_type,outcome_subtype,sex_upon_outcome,age_upon_outcome,has_name,age_upon_intake_day,age_upon_outcome_day,birth_sex,pure_color,sex_changed,intake_times
0,A006100,Scamp,2014-03-07 14:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,2014-03-08 17:10:00,2007-07-09,Return to Owner,NaN,Neutered Male,6 years,1.0,2433.0,2434.0,Male,0,1.0,1
1,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,2014-12-20 16:35:00,2007-07-09,Return to Owner,NaN,Neutered Male,7 years,1.0,2720.0,2721.0,Male,0,1.0,2
3,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,NaT,NaT,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Male,0,NaN,3
4,A047759,Oreo,2014-04-02 15:55:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,10 years,Dachshund,Tricolor,2014-04-07 15:12:00,2004-04-02,Transfer,Partner,Neutered Male,10 years,1.0,3652.0,3657.0,Male,0,1.0,1
5,A134067,Bandit,2013-11-16 09:02:00,12034 Research Blvd in Austin (TX),Public Assist,Injured,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White,2013-11-16 11:54:00,1997-10-16,Return to Owner,NaN,Neutered Male,16 years,1.0,5875.0,5875.0,Male,0,1.0,1


In [36]:
df_clean.shape

(132508, 24)

In [37]:
df_clean.to_csv("../data/Austin_Animal_Center_Cleaned.csv", index=False)